###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
import pickle
from collections import deque
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
GNRC = geodict.GNRC
KY = geodict.KY
censusplaces = geodict.censusplaces

# American Community Survey 2017-2021 5 Year Estimates

In [3]:
#read in API key
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [4]:
#variable containing Census API key
api_key = keys_dict_2['CENSUS']

B19013_001E:median hh inceom

In [6]:
msas = ['34980', '17300']

In [7]:
data_appended = []
for msa in msas:
    url_str= 'https://api.census.gov/data/2021/acs/acs5?key='+api_key
    predicates= {}
    get_vars= ["NAME", "GEO_ID", 'B19013_001E',]
               
              # means of transportation to work by travel time to work 'B08134_001E']
               
               #'C08536_002E', 'C08536_003E', 'C08101_008E', 'C08101_015E']
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "metropolitan statistical area/micropolitan statistical area:{}".format(msa)
    data= requests.get(url_str, params= predicates)
    col_names = ['NAME','GEOID','Median hh income', 'GeoFIPS']
                 
                 
                 #'Aggregate Travel Time to Work Drove Alone', 'Aggregate Travel Time to Work Carpooled', 'Car truck or van drove alone', 'Car truck or van carpooled']
    data=pd.DataFrame(columns=col_names, data=data.json()[1:])
    data_appended.append(data) #append the current instance to the last
data_appended = pd.concat(data_appended) 
data = data_appended
print('Your API call is complete')

Your API call is complete


In [8]:
data

,NAME,GEOID,Median hh income,GeoFIPS
0,"Nashville-Davidson--Murfreesboro--Franklin, TN...",310M600US34980,72537,34980
0,"Clarksville, TN-KY Metro Area",310M600US17300,57963,17300


In [143]:
data_appended = []
for msa in msas:
    url_str= 'https://api.census.gov/data/2021/acs/acs1?key='+api_key
    predicates= {}
    get_vars= ["NAME", "GEO_ID", 'B08303_001E', 'B08303_002E', 'B08303_003E', 'B08303_004E', 'B08303_005E', 'B08303_006E', 'B08303_007E', 
               'B08303_008E', 'B08303_009E', 'B08303_010E', 'B08303_011E', 'B08303_012E', 'B08303_013E', 'B08135_001E', 'B99084_002E']
               
              # means of transportation to work by travel time to work 'B08134_001E']
               
               #'C08536_002E', 'C08536_003E', 'C08101_008E', 'C08101_015E']
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "metropolitan statistical area/micropolitan statistical area:{}".format(msa)
    data= requests.get(url_str, params= predicates)
    col_names = ['NAME','GEOID','Travel Time Total Series', 'Travel Time Less than 5 Minutes', 'Travel Time 5 to 9 Minutes', 
                 'Travel Time 10 to 14 Minutes', 'Travel Time 15 to 19 Minutes', 'Travel Time 20 to 24 Minutes', 'Travel Time 25 to 29 Minutes', 
                 'Travel Time 30 to 34 Minutes', 'Travel Time 35 to 39 Minutes','Travel Time 40 to 44 Minutes', 'Travel Time 45 to 59 Minutes', 
                 'Travel Time 60 to 89 Minutes', 'Travel Time 90 or More Minutes', 'Aggregate Travel Time to Work', 'Those Traveling to Work', 'GeoFIPS']
                 
                 
                 #'Aggregate Travel Time to Work Drove Alone', 'Aggregate Travel Time to Work Carpooled', 'Car truck or van drove alone', 'Car truck or van carpooled']
    data=pd.DataFrame(columns=col_names, data=data.json()[1:])
    data_appended.append(data) #append the current instance to the last
data_appended = pd.concat(data_appended) 
data = data_appended
print('Your API call is complete')

Your API call is complete


In [144]:
data = data.drop(columns = ['GEOID', 'GeoFIPS'])
data = data.set_index('NAME')
cols = data.columns
data[cols] = data[cols].astype(float)
data = data.transpose()

In [145]:
data['Combined MSAs'] = data['Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area'] + data['Clarksville, TN-KY Metro Area']
#data = data[['Combined MSAs']]
data = data.transpose()
cols = data.columns
data[cols] = data[cols].astype(float)

In [146]:
thelist = [data['Travel Time Less than 5 Minutes'], data['Travel Time 5 to 9 Minutes']]
data['10 or Less'] = sum(thelist)
data['% 10 or Less'] = (data['10 or Less']/data['Travel Time Total Series'])*100 

data['Less Than 15'] = data['10 or Less'] + data['Travel Time 10 to 14 Minutes']
data['% Less Than 15'] = (data['Less Than 15']/data['Travel Time Total Series'])*100 

thelist = [data['Travel Time 15 to 19 Minutes'], 
          data['Travel Time 20 to 24 Minutes'], data['Travel Time 25 to 29 Minutes']]
data['15 to 30'] = sum(thelist)
data['% 15 to 30'] = (data['15 to 30']/data['Travel Time Total Series'])*100 

thelist = [data['Travel Time 30 to 34 Minutes'], data['Travel Time 35 to 39 Minutes'], 
          data['Travel Time 40 to 44 Minutes']]
data['30 to 45'] = sum(thelist)
data['% 30 to 45'] = (data['30 to 45']/data['Travel Time Total Series'])*100 


data['45 to 1 Hour'] = data['Travel Time 45 to 59 Minutes']
data['% 45 to 1 Hour'] = (data['45 to 1 Hour']/data['Travel Time Total Series'])*100 
data['More Than 1 Hour'] = data['Travel Time 60 to 89 Minutes'] + data['Travel Time 90 or More Minutes']
data['% More Than 1 Hour'] = (data['More Than 1 Hour']/data['Travel Time Total Series'])*100 

# aggtime = [data['Aggregate Travel Time to Work Drove Alone'], data['Aggregate Travel Time to Work Carpooled']]
# aggtime = sum(aggtime)
# notfromhome = [data['Car truck or van drove alone'], data['Car truck or van carpooled']]
# notfromhome = sum(notfromhome)

# data['Time In Car Traveling to Work Per Capita'] = aggtime/notfromhome

data['Time Traveling to Work Per Capita'] = data['Aggregate Travel Time to Work']/data['Those Traveling to Work']

cols = ['Less Than 15', '% Less Than 15', '15 to 30', '% 15 to 30', '30 to 45', '% 30 to 45', '45 to 1 Hour', '% 45 to 1 Hour', 'More Than 1 Hour', 
        '% More Than 1 Hour', 'Time Traveling to Work Per Capita']
data = data[cols]

In [147]:
twentytwentyone = data
twentytwentyone['Year'] = '2021'
twentytwentyone

,Less Than 15,% Less Than 15,15 to 30,% 15 to 30,30 to 45,% 30 to 45,45 to 1 Hour,% 45 to 1 Hour,More Than 1 Hour,% More Than 1 Hour,Time Traveling to Work Per Capita,Year
NAME,,,,,,,,,,,,
"Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",193327.0,23.641998,300533.0,36.752241,191411.0,23.407690,78189.0,9.561749,54267.0,6.636322,26.230453,2021
"Clarksville, TN-KY Metro Area",36453.0,27.381095,51459.0,38.652615,24910.0,18.710753,7907.0,5.939218,12403.0,9.316318,25.427621,2021
Combined MSAs,229780.0,24.165518,351992.0,37.018317,216321.0,22.750061,86096.0,9.054550,66670.0,7.011555,26.118047,2021


In [148]:
#compare from subject table
data_appended = []
for msa in msas:
    url_str= 'https://api.census.gov/data/2021/acs/acs1/subject?key='+api_key
    predicates= {}
    get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E']
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "metropolitan statistical area/micropolitan statistical area:{}".format(msa)
    data= requests.get(url_str, params= predicates)
    col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'GeoFIPS']
                 
                 
                 #'Aggregate Travel Time to Work Drove Alone', 'Aggregate Travel Time to Work Carpooled', 'Car truck or van drove alone', 'Car truck or van carpooled']
    data=pd.DataFrame(columns=col_names, data=data.json()[1:])
    data_appended.append(data) #append the current instance to the last
data_appended = pd.concat(data_appended) 
data = data_appended
print('Your API call is complete')

Your API call is complete


In [149]:
data

,NAME,GEO_ID,Mean Travel Time to Work,GeoFIPS
0,"Nashville-Davidson--Murfreesboro--Franklin, TN...",310M600US34980,26.2,34980
0,"Clarksville, TN-KY Metro Area",310M600US17300,25.4,17300


In [150]:
data_appended = []
for msa in msas:
    url_str= 'https://api.census.gov/data/2015/acs/acs1?key='+api_key
    predicates= {}
    get_vars= ["NAME", "GEO_ID", 'B08303_001E', 'B08303_002E', 'B08303_003E', 'B08303_004E', 'B08303_005E', 'B08303_006E', 'B08303_007E', 
               'B08303_008E', 'B08303_009E', 'B08303_010E', 'B08303_011E', 'B08303_012E', 'B08303_013E', 'B08135_001E', 'B99084_002E']
               
              # means of transportation to work by travel time to work 'B08134_001E']
               
               #'C08536_002E', 'C08536_003E', 'C08101_008E', 'C08101_015E']
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "metropolitan statistical area/micropolitan statistical area:{}".format(msa)
    data= requests.get(url_str, params= predicates)
    col_names = ['NAME','GEOID','Travel Time Total Series', 'Travel Time Less than 5 Minutes', 'Travel Time 5 to 9 Minutes', 
                 'Travel Time 10 to 14 Minutes', 'Travel Time 15 to 19 Minutes', 'Travel Time 20 to 24 Minutes', 'Travel Time 25 to 29 Minutes', 
                 'Travel Time 30 to 34 Minutes', 'Travel Time 35 to 39 Minutes','Travel Time 40 to 44 Minutes', 'Travel Time 45 to 59 Minutes', 
                 'Travel Time 60 to 89 Minutes', 'Travel Time 90 or More Minutes', 'Aggregate Travel Time to Work', 'Those Traveling to Work', 'GeoFIPS']
                 
                 
                 #'Aggregate Travel Time to Work Drove Alone', 'Aggregate Travel Time to Work Carpooled', 'Car truck or van drove alone', 'Car truck or van carpooled']
    data=pd.DataFrame(columns=col_names, data=data.json()[1:])
    data_appended.append(data) #append the current instance to the last
data_appended = pd.concat(data_appended) 
data = data_appended
print('Your API call is complete')

Your API call is complete


In [151]:
data = data.drop(columns = ['GEOID', 'GeoFIPS'])
data = data.set_index('NAME')
cols = data.columns
data[cols] = data[cols].astype(float)
data = data.transpose()

In [152]:
data['Combined MSAs'] = data['Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area'] + data['Clarksville, TN-KY Metro Area']
#data = data[['Combined MSAs']]
data = data.transpose()
cols = data.columns
data[cols] = data[cols].astype(float)

In [153]:
thelist = [data['Travel Time Less than 5 Minutes'], data['Travel Time 5 to 9 Minutes']]
data['10 or Less'] = sum(thelist)
data['% 10 or Less'] = (data['10 or Less']/data['Travel Time Total Series'])*100 

data['Less Than 15'] = data['10 or Less'] + data['Travel Time 10 to 14 Minutes']
data['% Less Than 15'] = (data['Less Than 15']/data['Travel Time Total Series'])*100 

thelist = [data['Travel Time 15 to 19 Minutes'], 
          data['Travel Time 20 to 24 Minutes'], data['Travel Time 25 to 29 Minutes']]
data['15 to 30'] = sum(thelist)
data['% 15 to 30'] = (data['15 to 30']/data['Travel Time Total Series'])*100 

thelist = [data['Travel Time 30 to 34 Minutes'], data['Travel Time 35 to 39 Minutes'], 
          data['Travel Time 40 to 44 Minutes']]
data['30 to 45'] = sum(thelist)
data['% 30 to 45'] = (data['30 to 45']/data['Travel Time Total Series'])*100 


data['45 to 1 Hour'] = data['Travel Time 45 to 59 Minutes']
data['% 45 to 1 Hour'] = (data['45 to 1 Hour']/data['Travel Time Total Series'])*100 
data['More Than 1 Hour'] = data['Travel Time 60 to 89 Minutes'] + data['Travel Time 90 or More Minutes']
data['% More Than 1 Hour'] = (data['More Than 1 Hour']/data['Travel Time Total Series'])*100 

# aggtime = [data['Aggregate Travel Time to Work Drove Alone'], data['Aggregate Travel Time to Work Carpooled']]
# aggtime = sum(aggtime)
# notfromhome = [data['Car truck or van drove alone'], data['Car truck or van carpooled']]
# notfromhome = sum(notfromhome)

# data['Time In Car Traveling to Work Per Capita'] = aggtime/notfromhome

data['Time Traveling to Work Per Capita'] = data['Aggregate Travel Time to Work']/data['Those Traveling to Work']

cols = ['Less Than 15', '% Less Than 15', '15 to 30', '% 15 to 30', '30 to 45', '% 30 to 45', '45 to 1 Hour', '% 45 to 1 Hour', 'More Than 1 Hour', 
        '% More Than 1 Hour', 'Time Traveling to Work Per Capita']
data = data[cols]

In [154]:
twentyfifteen = data
twentyfifteen['Year'] = '2015'
twentyfifteen

,Less Than 15,% Less Than 15,15 to 30,% 15 to 30,30 to 45,% 30 to 45,45 to 1 Hour,% 45 to 1 Hour,More Than 1 Hour,% More Than 1 Hour,Time Traveling to Work Per Capita,Year
NAME,,,,,,,,,,,,
"Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",176955.0,20.432258,328304.0,37.907898,200920.0,23.199397,88451.0,10.213069,71427.0,8.247379,27.412624,2015
"Clarksville, TN-KY Metro Area",37709.0,30.458136,49067.0,39.632166,22238.0,17.961973,6532.0,5.275996,8260.0,6.671728,23.043108,2015
Combined MSAs,214664.0,21.686233,377371.0,38.123559,223158.0,22.544332,94983.0,9.595570,79687.0,8.050306,26.866112,2015


In [155]:
data_appended = []
for msa in msas:
    url_str= 'https://api.census.gov/data/2010/acs/acs1?key='+api_key
    predicates= {}
    get_vars= ["NAME", "GEO_ID", 'B08303_001E', 'B08303_002E', 'B08303_003E', 'B08303_004E', 'B08303_005E', 'B08303_006E', 'B08303_007E', 
               'B08303_008E', 'B08303_009E', 'B08303_010E', 'B08303_011E', 'B08303_012E', 'B08303_013E', 'B08135_001E', 'B99084_002E']
               
              # means of transportation to work by travel time to work 'B08134_001E']
               
               #'C08536_002E', 'C08536_003E', 'C08101_008E', 'C08101_015E']
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "metropolitan statistical area/micropolitan statistical area:{}".format(msa)
    data= requests.get(url_str, params= predicates)
    col_names = ['NAME','GEOID','Travel Time Total Series', 'Travel Time Less than 5 Minutes', 'Travel Time 5 to 9 Minutes', 
                 'Travel Time 10 to 14 Minutes', 'Travel Time 15 to 19 Minutes', 'Travel Time 20 to 24 Minutes', 'Travel Time 25 to 29 Minutes', 
                 'Travel Time 30 to 34 Minutes', 'Travel Time 35 to 39 Minutes','Travel Time 40 to 44 Minutes', 'Travel Time 45 to 59 Minutes', 
                 'Travel Time 60 to 89 Minutes', 'Travel Time 90 or More Minutes', 'Aggregate Travel Time to Work', 'Those Traveling to Work', 'GeoFIPS']
                 
                 
                 #'Aggregate Travel Time to Work Drove Alone', 'Aggregate Travel Time to Work Carpooled', 'Car truck or van drove alone', 'Car truck or van carpooled']
    data=pd.DataFrame(columns=col_names, data=data.json()[1:])
    data_appended.append(data) #append the current instance to the last
data_appended = pd.concat(data_appended) 
data = data_appended
print('Your API call is complete')

Your API call is complete


In [156]:
data = data.drop(columns = ['GEOID', 'GeoFIPS'])
data = data.set_index('NAME')
cols = data.columns
data[cols] = data[cols].astype(float)
data = data.transpose()

In [157]:
data['Combined MSAs'] = data['Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area'] + data['Clarksville, TN-KY Metro Area']
#data = data[['Combined MSAs']]
data = data.transpose()
cols = data.columns
data[cols] = data[cols].astype(float)

In [158]:
thelist = [data['Travel Time Less than 5 Minutes'], data['Travel Time 5 to 9 Minutes']]
data['10 or Less'] = sum(thelist)
data['% 10 or Less'] = (data['10 or Less']/data['Travel Time Total Series'])*100 

data['Less Than 15'] = data['10 or Less'] + data['Travel Time 10 to 14 Minutes']
data['% Less Than 15'] = (data['Less Than 15']/data['Travel Time Total Series'])*100 

thelist = [data['Travel Time 15 to 19 Minutes'], 
          data['Travel Time 20 to 24 Minutes'], data['Travel Time 25 to 29 Minutes']]
data['15 to 30'] = sum(thelist)
data['% 15 to 30'] = (data['15 to 30']/data['Travel Time Total Series'])*100 

thelist = [data['Travel Time 30 to 34 Minutes'], data['Travel Time 35 to 39 Minutes'], 
          data['Travel Time 40 to 44 Minutes']]
data['30 to 45'] = sum(thelist)
data['% 30 to 45'] = (data['30 to 45']/data['Travel Time Total Series'])*100 


data['45 to 1 Hour'] = data['Travel Time 45 to 59 Minutes']
data['% 45 to 1 Hour'] = (data['45 to 1 Hour']/data['Travel Time Total Series'])*100 
data['More Than 1 Hour'] = data['Travel Time 60 to 89 Minutes'] + data['Travel Time 90 or More Minutes']
data['% More Than 1 Hour'] = (data['More Than 1 Hour']/data['Travel Time Total Series'])*100 

# aggtime = [data['Aggregate Travel Time to Work Drove Alone'], data['Aggregate Travel Time to Work Carpooled']]
# aggtime = sum(aggtime)
# notfromhome = [data['Car truck or van drove alone'], data['Car truck or van carpooled']]
# notfromhome = sum(notfromhome)

# data['Time In Car Traveling to Work Per Capita'] = aggtime/notfromhome

data['Time Traveling to Work Per Capita'] = data['Aggregate Travel Time to Work']/data['Those Traveling to Work']

cols = ['Less Than 15', '% Less Than 15', '15 to 30', '% 15 to 30', '30 to 45', '% 30 to 45', '45 to 1 Hour', '% 45 to 1 Hour', 'More Than 1 Hour', 
        '% More Than 1 Hour', 'Time Traveling to Work Per Capita']
data = data[cols]

In [159]:
twentyten = data
twentyten['Year'] = '2010'
twentyten

,Less Than 15,% Less Than 15,15 to 30,% 15 to 30,30 to 45,% 30 to 45,45 to 1 Hour,% 45 to 1 Hour,More Than 1 Hour,% More Than 1 Hour,Time Traveling to Work Per Capita,Year
NAME,,,,,,,,,,,,
"Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",155600.0,22.644848,262281.0,38.170395,160931.0,23.420682,68105.0,9.911487,40215.0,5.852587,26.100160,2010
"Clarksville, TN-KY Metro Area",31627.0,29.749509,43604.0,41.015511,18580.0,17.477025,6581.0,6.190328,5919.0,5.567627,22.971706,2010
Combined MSAs,187227.0,23.596780,305885.0,38.551604,179511.0,22.624309,74686.0,9.412900,46134.0,5.814406,25.680988,2010


In [160]:
dfs = [twentytwentyone, twentyfifteen, twentyten]
data = pd.concat(dfs)

In [161]:
data

,Less Than 15,% Less Than 15,15 to 30,% 15 to 30,30 to 45,% 30 to 45,45 to 1 Hour,% 45 to 1 Hour,More Than 1 Hour,% More Than 1 Hour,Time Traveling to Work Per Capita,Year
NAME,,,,,,,,,,,,
"Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",193327.0,23.641998,300533.0,36.752241,191411.0,23.407690,78189.0,9.561749,54267.0,6.636322,26.230453,2021
"Clarksville, TN-KY Metro Area",36453.0,27.381095,51459.0,38.652615,24910.0,18.710753,7907.0,5.939218,12403.0,9.316318,25.427621,2021
Combined MSAs,229780.0,24.165518,351992.0,37.018317,216321.0,22.750061,86096.0,9.054550,66670.0,7.011555,26.118047,2021
"Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",176955.0,20.432258,328304.0,37.907898,200920.0,23.199397,88451.0,10.213069,71427.0,8.247379,27.412624,2015
"Clarksville, TN-KY Metro Area",37709.0,30.458136,49067.0,39.632166,22238.0,17.961973,6532.0,5.275996,8260.0,6.671728,23.043108,2015
Combined MSAs,214664.0,21.686233,377371.0,38.123559,223158.0,22.544332,94983.0,9.595570,79687.0,8.050306,26.866112,2015
"Nashville-Davidson--Murfreesboro--Franklin, TN Metro Area",155600.0,22.644848,262281.0,38.170395,160931.0,23.420682,68105.0,9.911487,40215.0,5.852587,26.100160,2010
"Clarksville, TN-KY Metro Area",31627.0,29.749509,43604.0,41.015511,18580.0,17.477025,6581.0,6.190328,5919.0,5.567627,22.971706,2010
Combined MSAs,187227.0,23.596780,305885.0,38.551604,179511.0,22.624309,74686.0,9.412900,46134.0,5.814406,25.680988,2010


In [162]:
data.to_csv('traveltimetowork.csv')